In [4]:
#Dependencies
import requests
import json
from api_keys import api_key_FBI
import pandas as pd

In [89]:
# the abbreviation of every state.
state_codes = pd.DataFrame({"Abbreviation":['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT',"DC",
                'DE', 'FL',"FS", 'GA',"GM", 'HI', 'IA',"ID", 'IL', 'IN',
                'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 
                'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 
                'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 
                'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 
                'TX', 'UT', 'VT', 'VA', 'WA', "VI",'WV', 'WI', 'WY']})

data_homicide=state_codes.copy()
data_homicide["Total Homicide"]=""
data_homicide.head()



,Abbreviation,Total Homicide
0,AL,
1,AK,
2,AZ,
3,AR,
4,CA,


# INFORMATION OF FBI HOMICIDE OF 2021

In [90]:
for index,row in data_homicide.iterrows():
    try:
        #with this we get the information of FBI API
        state=row["Abbreviation"]
        base_url=f"https://api.usa.gov/crime/fbi/cde/shr/state/{state}/victim/sex?from=2021&to=2021&API_KEY={api_key_FBI}"
        #print(base_url)
        response = requests.get(base_url)
        response= response.json()

        data_homicide.loc[index,"Total Homicide"]=response["data"][0]["Unknown"]+response["data"][0]["Male"]+response["data"][0]["Female"]
    except:
        data_homicide.loc[index,"Total Homicide"]=""

data_homicide.head()
    


,Abbreviation,Total Homicide
0,AL,309
1,AK,18
2,AZ,190
3,AR,321
4,CA,123


# INFORMATION OF COST LIVING

In [76]:
cost_living_data=pd.read_csv("resources/cost-of-living-index-by-state-[updated-june-2023].csv")
cost_living_data["Rank"]=cost_living_data["2023"]
cost_living_data["State"]=cost_living_data["state"]
cost_living_data=cost_living_data.loc[:,["State","Rank"]]

cost_living_data.head()


,State,Rank
0,Hawaii,179.0
1,District of Columbia,148.7
2,Massachusetts,148.4
3,California,134.5
4,New York,125.1


# INFORMATION of HEALTH COST

In [39]:
health_data=pd.read_csv("resources/data-hGOWY.csv")
health_data.head()

,Rank,State,Score,% of adults reporting unmet need for mental health treatment in the past year who did not receive care because of cost,Health Care Spending Per Capita
0,1,South Dakota,100.00,47.10%,"$11,736"
1,2,Louisiana,86.69,32.00%,"$9,796"
2,3,West Virginia,82.31,39.50%,"$12,019"
3,4,Florida,79.51,51.90%,"$9,501"
4,5,Wyoming,78.63,64.70%,"$10,296"


In [41]:
states_USA=pd.read_csv("resources/states.csv")
states_USA.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [86]:
resume=pd.merge(states_USA,health_data[["Rank","State"]],on="State",how="inner")
resume.head()

,State,Abbreviation,Rank
0,Alabama,AL,28
1,Alaska,AK,12
2,Arizona,AZ,13
3,Arkansas,AR,33
4,California,CA,32


In [88]:
resume=pd.merge(resume,cost_living_data,on="State",how="inner")
resume.head()

,State,Abbreviation,Rank_x,Rank_y
0,Alabama,AL,28,88.8
1,Alaska,AK,12,124.4
2,Arizona,AZ,13,107.2
3,Arkansas,AR,33,90.3
4,California,CA,32,134.5


In [91]:
resume=pd.merge(resume,data_homicide,on="Abbreviation",how="inner")
resume.head()

,State,Abbreviation,Rank_x,Rank_y,Total Homicide
0,Alabama,AL,28,88.8,309
1,Alaska,AK,12,124.4,18
2,Arizona,AZ,13,107.2,190
3,Arkansas,AR,33,90.3,321
4,California,CA,32,134.5,123
